In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy
import copy
import matplotlib
import os
from matplotlib import pyplot as plt
import seaborn as sns
import torch.nn.functional as F
sns.set(style="ticks")
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import numpy as np
from utils import Str2id
from collections import Counter
from utils import NodeClassifier,get_word_embeddings
from tqdm import tqdm_notebook as tqdm
# from tqdm import tqdm
from tqdm import tnrange
from datasets import NormalEdgeDataset, NodeTextEdgeDataset, MultipleLableDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
# %config InlineBackend.figure_format = 'retina'
from scoring import evaluate
import pickle
from model import NodeRepresentation, SemiSkipGram
import time

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def collate_fn(data):
    u = [d[0] for d in data]
    v = [d[1] for d in data]
    neg_v = []
    for d in data:
        neg_v += d[2]
    return u, v, numpy.array(neg_v)

In [15]:
# use_cuda=torch.cuda.is_available()
batch_size=100
neg_sampling_size=10
emb_size=256

In [5]:
base_data_path="/mnt/store1/plus1lab/multilabel-data"
base_model_data_path="/mnt/store1/plus1lab/multilabel-model"

In [24]:
dataset="v1_aminer_5k"
ne_model='deepwalk'

In [25]:
edge_list_path = "%s/%s_5/original-edgelist.txt" % (base_data_path, dataset)
stats_file = "%s/%s_5/stats.txt" % (base_data_path, dataset)
node_labels_file = "%s/%s_5/node-labels.txt" % (base_data_path, dataset)
original_deepwalk_path = "%s/deepwalk-node/%s_5/original.walks" % (base_model_data_path,
    dataset)

In [26]:
node_map = Str2id()
label_map = Str2id()
if ne_model == 'deepwalk':
    edge_dataset = NormalEdgeDataset(
        node_map,
        node_map,
        original_deepwalk_path,
        neg_sampling_size,
        from_random_walk=True,
        window_size=10)
elif ne_model == 'skip-gram':
    edge_dataset = NormalEdgeDataset(node_map, node_map, edge_list_path,
                                     neg_sampling_size)
else:
    raise Exception('No such network embedding model')
edge_dataset.negative_sampling1()

7232it [00:02, 3210.55it/s]/home/hcchen/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
20150it [00:05, 3485.57it/s]


In [27]:
edge_label_ratio=5
edge_train_prob=1.0
edge_label_train_prob=0.2

In [29]:
print("edge_label_train_prob", edge_label_train_prob)
print("batch_size", batch_size)
print("neg_sampling_size", neg_sampling_size)
print("dataset", dataset)
print("ne_model", ne_model)
filename = str(time.time())
print("filename", filename)

# for edge_label_train_prob in [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]:
for edge_label_train_prob in [0.0]:
# for edge_label_ratio in [1, 5, 10]:
# for emb_size in [16, 32, 64, 128, 256, 512]:
# for emb_size in [256]:
    print("edge_label_ratio", edge_label_ratio)
    print("edge_train_prob", edge_train_prob)
    print("emb_size", emb_size)

    edge_labels_path = "%s/aegcn/%s_%d/label-mat.txt" % (base_model_data_path,
                                                         dataset,
                                                         edge_label_ratio)
    all_scores = []
    node_map.freeze = True
    total_label_count = int(open(stats_file).readline().strip().split(" ")[2])
    edge_label_dataset = MultipleLableDataset(
        node_map,
        node_map,
        edge_labels_path,
        total_label_count,
        ratio=1.0,
        use_all_zeros=False)
    print(len(node_map.str2id_dict))
    use_cuda = False
    print(len(edge_label_dataset))

    torch.cuda.manual_seed_all(13)
    torch.manual_seed(13)
    model = SemiSkipGram(emb_size, total_label_count, use_cuda)
    node_representation = NodeRepresentation(
        len(node_map.str2id_dict), emb_size, use_cuda)
    node_optimizer = torch.optim.SGD(node_representation.parameters(), 1)
    model_optimizer = torch.optim.SGD(model.parameters(), 1)

    edge_loader = DataLoader(
        edge_dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=True,
        drop_last=True)
    edge_label_loader = DataLoader(
        edge_label_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True)

    lambda0 = lambda epoch: max(0.00001, 0.05 / (1.0 + epoch * batch_size / 10000 * 0.01))
    scheduler0 = optim.lr_scheduler.LambdaLR(
        node_optimizer, lr_lambda=[lambda0])
    lambda1 = lambda epoch: max(0.00001, 0.05 / (1.0 + epoch * batch_size / 10000 * 0.01))
    scheduler1 = optim.lr_scheduler.LambdaLR(
        model_optimizer, lr_lambda=[lambda1])

    edge_errs = [0]
    edge_label_errs = [0]
    edge_loader_train = iter(edge_loader)
    edge_label_loader_train = iter(edge_label_loader)
    batch_process_bar = tqdm(range(int(100000)))

    for idx in batch_process_bar:
        directed = True
        if idx % 5000 == 0 or (idx < 5000 and idx % 5000 == 0):

            node_representation.save_embedding(node_map, filename, False)
            score = evaluate(
                filename, node_labels_file, stats_file, num_shuffles=1, use_std_out=True)
            all_scores.append([idx, score])
            os.remove(filename)
        if numpy.random.rand() < edge_train_prob:
            try:
                uids, vids, neg_vids = edge_loader_train.next()
            except:
                edge_loader_train = iter(edge_loader)
                edge_dataset.negative_sampling1()
                uids, vids, neg_vids = edge_loader_train.next()
            node_optimizer.zero_grad()
            emb_u = node_representation.forward(
                uids, is_start=True, directed=directed)
            emb_v = node_representation.forward(
                vids, is_start=False, directed=directed)
            neg_emb_v = node_representation.forward(
                neg_vids, is_start=False, directed=directed)
            neg_emb_v = neg_emb_v.view(batch_size, -1, emb_size)
            err = model.stucture_loss(emb_u, emb_v, neg_emb_v)
            # edge_errs.append(err.cpu().data.numpy()[0])
            err.backward()
            node_optimizer.step()
            scheduler0.step()

        if numpy.random.rand() < edge_label_train_prob:
            try:
                uids, vids, edge_labels = edge_label_loader_train.next()
            except:
                edge_label_loader_train = iter(edge_label_loader)
                uids, vids, edge_labels = edge_label_loader_train.next()

            edge_labels = Variable(edge_labels.type(torch.FloatTensor))
            node_optimizer.zero_grad()
            model_optimizer.zero_grad()
            emb_u = node_representation.forward(
                uids, is_start=True, directed=directed)
            emb_v = node_representation.forward(
                vids, is_start=False, directed=directed)
            err = model.edge_label_loss(emb_u, emb_v, edge_labels)
            err.backward()
            model_optimizer.step()
            node_optimizer.step()
            # edge_label_errs.append(err.cpu().data.numpy()[0])
        batch_process_bar.set_postfix_str("Loss: %0.2f | %0.2f, lr1: %0.6f" % (
            edge_errs[-1],
            edge_label_errs[-1],
            node_optimizer.param_groups[0]["lr"],
        ))
    result_file_name = "../../results/our_model/%s/%s/%d/training_prob_%0.2f_%0.2f_batch_%d_dim_%d_neg_%d.pkl" % (
        dataset, ne_model, edge_label_ratio, edge_train_prob,
        edge_label_train_prob, batch_size, emb_size, neg_sampling_size)
    pickle.dump(all_scores, open(result_file_name, "wb"))

edge_label_train_prob 0.0
batch_size 100
neg_sampling_size 10
dataset v1_aminer_5k
ne_model deepwalk
filename 1534902302.492674
edge_label_ratio 5
edge_train_prob 1.0
emb_size 256
Use 1.00 labels
not count topk 0
2015
147


HBox(children=(IntProgress(value=0, max=100000), HTML(value='')))

2015 lines [00:00, 275536.22 lines/s]


Results, using embeddings of dimensionality 256
-------------------
Train percent: 0.05
Shuffle #1:    {'micro': 0.11000433087916847, 'accuracy': 0.004699738903394256, 'macro': 0.03447154983441913}
Average score: {'micro': 0.11000433087916847, 'accuracy': 0.004699738903394256, 'macro': 0.03447154983441913}
-------------------
Train percent: 0.1
Shuffle #1:    {'micro': 0.11861564262844945, 'accuracy': 0.0027563395810363835, 'macro': 0.03801074142944303}
Average score: {'micro': 0.11861564262844945, 'accuracy': 0.0027563395810363835, 'macro': 0.03801074142944303}
-------------------
Train percent: 0.2
Shuffle #1:    {'micro': 0.12575262343024257, 'accuracy': 0.0024813895781637717, 'macro': 0.038161649351920686}
Average score: {'micro': 0.12575262343024257, 'accuracy': 0.0024813895781637717, 'macro': 0.038161649351920686}
-------------------


/home/hcchen/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
2015 lines [00:00, 378873.11 lines/s]


...

/home/hcchen/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
2015 lines [00:00, 257049.26 lines/s]


Results, using embeddings of dimensionality 256
-------------------
Train percent: 0.05
Shuffle #1:    {'micro': 0.18945848375451263, 'accuracy': 0.044386422976501305, 'macro': 0.1445526083737005}
Average score: {'micro': 0.18945848375451263, 'accuracy': 0.044386422976501305, 'macro': 0.1445526083737005}
-------------------
Train percent: 0.1
Shuffle #1:    {'micro': 0.24521712723959915, 'accuracy': 0.05733186328555678, 'macro': 0.21665748632672688}
Average score: {'micro': 0.24521712723959915, 'accuracy': 0.05733186328555678, 'macro': 0.21665748632672688}
-------------------
Train percent: 0.2
Shuffle #1:    {'micro': 0.3116663852777309, 'accuracy': 0.08002481389578164, 'macro': 0.2882478994813832}
Average score: {'micro': 0.3116663852777309, 'accuracy': 0.08002481389578164, 'macro': 0.2882478994813832}
-------------------


/home/hcchen/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
2015 lines [00:00, 7497.04 lines/s]


Results, using embeddings of dimensionality 256
-------------------
Train percent: 0.05
Shuffle #1:    {'micro': 0.20043509789702682, 'accuracy': 0.045953002610966055, 'macro': 0.16200196265572256}
Average score: {'micro': 0.20043509789702682, 'accuracy': 0.045953002610966055, 'macro': 0.16200196265572256}
-------------------
Train percent: 0.1
Shuffle #1:    {'micro': 0.2505736576411198, 'accuracy': 0.05181918412348401, 'macro': 0.22262105184045808}
Average score: {'micro': 0.2505736576411198, 'accuracy': 0.05181918412348401, 'macro': 0.22262105184045808}
-------------------
Train percent: 0.2
Shuffle #1:    {'micro': 0.31435431315383294, 'accuracy': 0.06327543424317618, 'macro': 0.2843042501817429}
Average score: {'micro': 0.31435431315383294, 'accuracy': 0.06327543424317618, 'macro': 0.2843042501817429}
-------------------


/home/hcchen/anaconda3/envs/deeplearning/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
